# Text and Speech Analysis with Cognitive Services


## Using the Text Analyics Cognitive Service

Microsoft Cognitive Services includes a Text Analytics service that encapsulates  sophisticated techniques for ascertaining meaning from text.

1. In Azure Portal navigate to Text Analytics service you created during the lab setup.
2. In the blade for your Text Analytics service, click **Keys** and then copy **Key 1** to the clipboard and paste it into the **textKey** variable below. 
3. Extract URI from **Endpoint** and paste it into the **textAnalyticsURI** variable.
3. Run the cell below to assign the variables.

In [ ]:
textAnalyticsURI = '<your URI>'
textKey = '<your key>'

### Load sample data

In [ ]:
# Use Curl to get a document from GitHub and open it
!curl https://raw.githubusercontent.com/MicrosoftLearning/AI-Introduction/master/files/Moon.txt -o Moon.txt
doc1 = open("Moon.txt", "r")
# Read the document and print its contents
doc1Txt = doc1.read()
print(doc1Txt)

# Get a second document, normalize it, and remove stop words
!curl https://raw.githubusercontent.com/MicrosoftLearning/AI-Introduction/master/files/Gettysburg.txt -o Gettysburg.txt
doc2 = open("Gettysburg.txt", "r")
doc2Txt = doc2.read()
print (doc2Txt)

### Call the Text Analytics Service to Determine Key Phrases in the Documents
One of the methods provided by the Text Analytics service is the ability to extract a list of key phrases from text documents, which give an insight into the core topics discussed in the document.

Run the following cell to call the **keyPhrases** method of the Text Analytics service and extract the key phrases for the text documents you have loaded so far in this notebook.

In [ ]:
import http.client, urllib.request, urllib.parse, urllib.error, base64, json, urllib

# Define the request headers.
headers = {
    'Content-Type': 'application/json',
    'Ocp-Apim-Subscription-Key': textKey,
    'Accept': 'application/json'
}

# Define the parameters
params = urllib.parse.urlencode({
})

# Define the request body
body = {
  "documents": [
    {
      "language": "en",
      "id": "1",
      "text": doc1Txt
    },
    {
          "language": "en",
          "id": "2",
          "text": doc2Txt
    }
  ]
}

try:
    # Execute the REST API call and get the response.
    conn = http.client.HTTPSConnection(textAnalyticsURI)
    conn.request("POST", "/text/analytics/v2.0/keyPhrases?%s" % params, str(body), headers)
    response = conn.getresponse()
    data = response.read().decode("UTF-8")

    # 'data' contains the JSON data. The following formats the JSON data for display.
    parsed = json.loads(data)
    for document in parsed['documents']:
        print("Document " + document["id"] + " key phrases:")
        for phrase in document['keyPhrases']:
            print("  " + phrase)
        print("---------------------------")
    conn.close()

except Exception as e:
    print('Error:')
    print(e)

From these key phrases, it's reasonably clear that the first document is about freedom and nationhood, while the second is about software services for AI.

### Perform Sentiment Analysis
Another common AI requirement is to determine the sentiment associated with some text. For example, you might analyze tweets that include your organization's twitter handle to determine if they are positive or negative.

Run the cell below to use the **sentiment** method of the Text Analytics service to discern the sentiment of two sentences.

In [ ]:
body = {
  "documents": [
    {
      "language": "en",
      "id": "1",
      "text": "Wow! cognitive services are fantastic."
    },
    {
      "language": "en",
      "id": "2",
      "text": "I hate it when computers don't understand me."
    }
  ]
}

try:
    conn = http.client.HTTPSConnection(textAnalyticsURI)
    conn.request("POST", "/text/analytics/v2.0/sentiment?%s" % params, str(body), headers)
    response = conn.getresponse()
    data = response.read().decode("UTF-8")
    parsed = json.loads(data)
    for document in parsed['documents']:
        sentiment = "negative"
        if document["score"] >= 0.5:
            sentiment = "positive"
        print("Document:" + document["id"] + " = " + sentiment)
    conn.close()
    
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))



## Working with Speech
So far, we've seen how analyze text; but increasingly AI systems enable humans to communicate with software services through speech recognition.

1. Navigate to your Speech service to open its blade.
2. In the blade for your Speech service, click **Keys** and then copy **Key 1** to the clipboard and paste it into the **speeckKey** variable assignment value in the cell below. 

In [ ]:
speechKey = '<your key>'

### Install the SpeechRecognition Package
You can call the Speech API by sending HTTP requests containing a JSON payload as you did for the Text Analytics API. However, the Speech API is supported in a pre-built Python package named **SpeechRecognition**, which makes it easier to use. You can find out more about this package at https://pypi.python.org/pypi/SpeechRecognition.

Run the following cell to install the **SpeechRecognition** package.

In [ ]:
# Install Custom Vision Service SDK  in the current Jupyter kernel
import sys
!{sys.executable} -m pip install SpeechRecognition


### Listen to the Speech
In this exercise, you will use a .wav audio file. To hear the speech you will analyze, run the cell below (this assumes you have audio capabilities in your computer!)

In [ ]:
import IPython

!curl https://raw.githubusercontent.com/MicrosoftLearning/AI-Introduction/master/files/RainSpain.wav -o RainSpain.wav

IPython.display.Audio('RainSpain.wav', autoplay=True)

### Call the  Speech Service to Transcribe the Audio
Hopefully you understood what was said in the audio file.

Let's see how the Speech API does!

In [ ]:
import speech_recognition as sr

# Read the audio file
r = sr.Recognizer()
with sr.AudioFile('RainSpain.wav') as source:
    audio = r.record(source)
    
# Alternative code to use mic input (only works in local Jupyter - not in Azure Notebooks)
# r = sr.Recognizer()
# with sr.Microphone() as source:
#     print("Say something!")
#     audio = r.listen(source)

# transcribe speech using the Bing Speech API
try:
    transcription = r.recognize_bing(audio, key=speechKey)
    print("Here's what I heard:")
    print('"' + transcription + '"')
    
except sr.UnknownValueError:
    print("The audio was unclear")
except sr.RequestError as e:
    print (e)
    print("Something went wrong :-(; {0}".format(e))


## Using the Language Understanding Intelligence Service (LUIS)
Increasingly, we expect computers to be able to use AI in order to understand spoken or typed commands in natural language. For example, we want to be able to say "switch on the light" or "put the light on", and have an AI-powered device understand the command and take appropriate action.

### Provision the Language Understanding Intelligence Service (LUIS)
The Microsoft cognitive services include the Language Understanding Intelligence Service (LUIS), which enables you to define *intents* that are applied to *entities* based on *utterances*.


### Create a LUIS App
To implement natural language understanding with LUIS, you must first create an app; and then add intents, utterances, and entities to define the commands you want the app to understand.
1. Open a new browser tab and navigate to https://www.luis.ai/.
2. Sign in using the Microsoft account associated with your Azure subscription. If this is the first time you have signed into LUIS, you may need to grant the app some permissions to access your account details, and then fill in some information and accept the terms of use.
3. If a message prompting you to complete a tutorial in which you will create a *Scheduler* app is displayed, close it (you can complete this tutorial later - for now, we'll focus on a simpler example).
4. Click **New App** and create a new app with the following settings:
  * **Name**: Simple Home Automation
  * **Culture**: English
  * **Description**: A basic home automation example
5. In the pane on the left, click **Intents**. Then click **Add Intent**, and add an intent with the name **Light On**.
6. In the **Utterances** page for the **Light On** intent, type "*switch the light on*" and press **Enter** to add this utterance to the list.
7. In the list of utterances, in the *switch the light on* utterance, hold the mouse over the word "light" so that the list shows the value *switch the [light] on*. Then in the **Search or create** box type *Light* and create a simple entity named **Light**.
8. In the pane on the left, click **Intents** and click **Add Intent**, to add a second intent with the name **Light Off**.
9. In the **Utterances** page for the **Light Off** intent, type "*switch the light off*" and press **Enter** to add this utterance to the list.
10. In the list of utterances, in the *switch the light on* utterance, hold the mouse over the word "light" so that the list shows the value *switch the [light] on*. Then click **[light]** select the *Light* entity you created previously.
11. At the top of the page, click **Train** to train the application
12. After the app has been trained, click **Test**, and then in the test pane, enter the following utterances and verify that they are correctly interpreted as commands for the *Light On* and *Light Off* intents respectively:
    * *turn on the light*
    * *put the light off*
13. IAt the top of the page, click **Publish**. Then click **Publish to production slot**.
14. After the app has been published, note the **Endpoint** URL that is generated for your app - this includes the location where you provisioned the service, your app ID, and the key assigned to the app.

### Consume the LUIS App
Now that you have published your LUIS app, you can consume it from a client application by making HTTP requests that include a query string. The query will be used to identify the most likely intent, which will be returned to the calling client as in JSON response.

Modify the **endpointUrl** variable declaration in the cell below to reflect the endpoint URL for your app. Then run the cell, and enter a command when prompted to call your service and interpret the command. The JSON response is shown with an appropriate image for each command.

Try the following commands:
* *Switch on the light*
* *Turn on the light*
* *Turn off the light*
* *Could you put the light on please?*

In [ ]:
%matplotlib inline
from matplotlib.pyplot import imshow
from PIL import Image
import requests
from io import BytesIO
import json 

# Set up API configuration
endpointUrl = "<your endpoint>"

# prompt for a command
command = input('Please enter a command: \n')

# Call the LUIS service and get the JSON response
endpoint = endpointUrl + command.replace(" ","+")
response = requests.get(endpoint)
data = json.loads(response.content.decode("UTF-8"))
print (data)

# Identify the top scoring intent
intent = data["topScoringIntent"]["intent"]
if (intent == "Light On"):
    img_url = 'https://raw.githubusercontent.com/MicrosoftLearning/AI-Introduction/master/files/LightOn.jpg'
elif (intent == "Light Off"):
    img_url = 'https://raw.githubusercontent.com/MicrosoftLearning/AI-Introduction/master/files/LightOff.jpg'
else:
    img_url = 'https://raw.githubusercontent.com/MicrosoftLearning/AI-Introduction/master/files/Dunno.jpg'

# Get the appropriate image and show it
response = requests.get(img_url)
img = Image.open(BytesIO(response.content))
imshow(img)


## Combining Speech Recognition and Natural Language Understanding
An obvious next step is to combine speech recognition with natural language understanding so that a spoken command can be interpreted and the appropriate action taken.

### Download Command Audio
Let's start by downloading and playing some spoken commands for our home automation system. Run the two cells under this to hear the commands.

In [ ]:
import IPython

# Get the "lights on" command
!curl https://raw.githubusercontent.com/MicrosoftLearning/AI-Introduction/master/files/LightOn.wav -o LightOn.wav
    
IPython.display.Audio('LightOn.wav', autoplay=True)

In [ ]:
# Get the "lights on" command
!curl https://raw.githubusercontent.com/MicrosoftLearning/AI-Introduction/master/files/LightOff.wav -o LightOff.wav
    
IPython.display.Audio('LightOff.wav', autoplay=True)

### Transcribe and Interpret the "Light On" Command
Now you can use the Bing Speech API to transcribe the command, and then use your LUIS app to interpret the command and take the appropriate action.

You should have already installed the **SpeechRecognition** package, set the **speechKey** for your Bing Speech service, and set the **apiUrl**, **appId**, and **appKey** for your LUIS app (if you have closed and re-opened the notebook, re-run the appropriate cells above to set these).

Run the cell below that to test the "Light On" command.

In [ ]:
%matplotlib inline
import speech_recognition as sr
from matplotlib.pyplot import imshow
from PIL import Image
import requests
from io import BytesIO
import json 

# Specify which audio file to use
audioFile = "LightOn.wav"

# Read the audio file
r = sr.Recognizer()
with sr.AudioFile(audioFile) as source:
    audio = r.record(source)

try:
    # transcribe speech using the Bing Speech API
    transcription = r.recognize_bing(audio, key=speechKey)
    
    # Call the LUIS service and get the JSON response
    endpoint = endpointUrl + transcription.replace(" ","+")
    response = requests.get(endpoint)
    data = json.loads(response.content.decode("UTF-8"))

    # Identify the top scoring intent
    intent = data["topScoringIntent"]["intent"]
    if (intent == "Light On"):
        img_url = 'https://raw.githubusercontent.com/MicrosoftLearning/AI-Introduction/master/files/LightOn.jpg'
    elif (intent == "Light Off"):
        img_url = 'https://raw.githubusercontent.com/MicrosoftLearning/AI-Introduction/master/files/LightOff.jpg'
    else:
        img_url = 'https://raw.githubusercontent.com/MicrosoftLearning/AI-Introduction/master/files/Dunno.jpg'

    # Get the appropriate image and show it
    response = requests.get(img_url)
    img = Image.open(BytesIO(response.content))
    imshow(img)
    
except sr.UnknownValueError:
    print("Bing Speech could not understand audio")
except sr.RequestError as e:
    print (e)
    print("Could not request results from the Bing Speech service; {0}".format(e))


### Transcribe and Interpret the "Light Off" Command
Modify the cell above to set the **audioFile** variable to the **LightOff.wav** audio file, and then run the cell again to test it